# Pre-Processing Datasets

## Introduction
In this notebook, we will be pre-processing the datasets that we will be using for the project. We will be using the following datasets:
1. flipkart_com-ecommerce_updated.csv

## Summary
Summary of Data Cleaning Process of FlipKart Dataset:
1. Changed Nan values in 'brand'to 'Unbranded'
2. removing rows with null values in `product_specifications` and `description`
3. Removing rows that has no `retail_price`, `discounted_price`, or `image`
4. Changing the `product_category_tree` column to a list of the main categories
5. Extracting the specifications from the `product_specifications` column
6. Removing the colors in the `brand` column (might need a second opinion on whether we should do this for the other columns)


In [1]:
# importing libraries
import pandas as pd

## cleaning

In [2]:
# this section of code is used to do basic cleaning of the flipkart dataset. 
# new dataset is saved as 'flipkart_cleaned.csv'
# with the new dataset, another set of synthetic data of orders are generated and saved as 'synthetic_v2.csv'

# reading flipkart orders data
filepath = 'oldData/flipkart_com-ecommerce_updated.csv'
flipkart = pd.read_csv(filepath)

# finding for null values
flipkart.isnull().sum() # sum of null values in each column

# changing column values

## change nan values in 'brand' to 'Unbranded'
flipkart['brand'] = flipkart['brand'].fillna('Unbranded')

## removing rows with null values in 'product_specifications' and 'description'
flipkart = flipkart.dropna(subset=['product_specifications', 'description', 'retail_price', 'discounted_price'])

## removing unnecessary columns
flipkart = flipkart.drop(columns = ['image', 'is_FK_Advantage_product', 'product_url'])


flipkart.isnull().sum()
# at this point there should not be any null values in the dataset. 




uniq_id                   0
product_name              0
product_category_tree     0
pid                       0
retail_price              0
discounted_price          0
discount                  0
description               0
overall_rating            0
brand                     0
product_specifications    0
dtype: int64

In [3]:
# dropping duplicates based on product id (pid)
flipkart = flipkart.drop_duplicates(subset=["pid"]) 

# changing all values to lowercase
flipkart = flipkart.apply(lambda col: col.str.lower() if col.dtypes == 'object' else col)

Since the product specifications are in the form of a dictionary, we will be defining a function to extract the specifications, removing any unnecessary information.

In [4]:
# changing product specifications
import re

def extract_specs(specs):
    pairs = re.findall(r'"key"=>"(.*?)", "value"=>"(.*?)"', specs)
    pairs_format = [f"{key}: {value}" for key, value in pairs]
    return ' '.join(pairs_format)

# applying the function to the product_specification column, changing all values to lowercase
flipkart['product_specifications'] = flipkart['product_specifications'].apply(extract_specs)
flipkart.head()

def remove_colors(brand):
    colors = [
        'red', 'green', 'blue', 'yellow', 'black', 'white', 'gray', 'cyan', 'magenta',
        'purple', 'pink', 'orange', 'brown', 'beige', 'maroon', 'navy', 'lime', 'olive',
        'chocolate', 'teal', 'silver', 'gold', 'azure', 'ivory', 'lavender', 'violet',
        'indigo', 'coral', 'salmon', 'khaki', 'orchid', 'turquoise', 'sienna', 'plum',
        'tan', 'fuchsia', 'burgundy', 'chartreuse', 'emerald', 'amethyst', 'ruby', 
        'sapphire', 'mint', 'peach', 'lime green', 'mustard', 'ochre', 'aquamarine', 
        'bisque', 'cadetblue', 'cornflowerblue', 'darkgoldenrod', 'darkolivegreen', 
        'darkorchid', 'deeppink', 'deepskyblue', 'dodgerblue', 'firebrick', 'forestgreen', 
        'gainsboro', 'ghostwhite', 'goldenrod', 'green yellow', 'honeydew', 'hotpink', 
        'indianred', 'ivory', 'khaki', 'lavender blush', 'lemon chiffon', 'light blue', 
        'light coral', 'light cyan', 'light goldenrod yellow', 'light green', 'light pink', 
        'light salmon', 'light sea green', 'light sky blue', 'light slate gray', 
        'light steel blue', 'light yellow', 'limegreen', 'linen', 'medium aquamarine', 
        'medium blue', 'medium orchid', 'medium purple', 'medium sea green', 
        'medium slate blue', 'medium spring green', 'medium turquoise', 'medium violet red', 
        'midnight blue', 'mint cream', 'misty rose', 'moccasin', 'navajo white', 'old lace', 
        'olive drab', 'orange red', 'orchid', 'pale goldenrod', 'pale green', 'pale turquoise', 
        'pale violet red', 'papaya whip', 'peach puff', 'peru', 'pink', 'plum', 'powder blue', 
        'rosy brown', 'royal blue', 'saddle brown', 'salmon', 'sandy brown', 
        'sea green', 'seashell', 'sienna', 'sky blue', 'slate blue', 'slate gray', 
        'snow', 'spring green', 'steel blue', 'tan', 'thistle', 'tomato', 'turquoise', 
        'violet', 'wheat', 'white smoke', 'yellow green', 'crimson', 'alice blue', 
        'antique white', 'aqua', 'aquamarine', 'blanched almond', 'blue violet', 'burlywood', 
        'cadet blue', 'chartreuse', 'chocolate', 'coral', 'cornflower blue', 'crimson', 
        'cyan', 'dark blue', 'dark cyan', 'dark grey', 'dark green', 'dark khaki', 
        'dark magenta', 'dark olive green', 'dark orange', 'dark orchid', 'dark red', 
        'dark salmon', 'dark sea green', 'dark slate blue', 'dark slate gray', 'dark turquoise', 
        'dark violet', 'deep pink', 'deep sky blue', 'dim gray', 'dim grey', 'dodger blue', 
        'firebrick', 'floral white', 'forest green', 'fuchsia', 'gainsboro', 'ghost white', 
        'gold', 'goldenrod', 'gray', 'green', 'green yellow', 'grey', 'honeydew', 'hot pink', 
        'indian red', 'indigo', 'ivory', 'khaki', 'lavender', 'lavender blush', 'lawn green', 
        'lemon chiffon', 'light blue', 'light coral', 'light cyan', 'light golden rod yellow', 
        'light gray', 'light grey', 'light green', 'light pink', 'light salmon', 
        'light sea green', 'light sky blue', 'light slate gray', 'light slate grey', 
        'light steel blue', 'light yellow', 'lime', 'lime green', 'linen', 'magenta', 
        'maroon', 'medium aquamarine', 'medium blue', 'medium orchid', 'medium purple', 
        'medium sea green', 'medium slate blue', 'medium spring green', 'medium turquoise', 
        'medium violet red', 'midnight blue', 'mint cream', 'misty rose', 'moccasin', 
        'navajo white', 'navy', 'old lace', 'olive', 'olive drab', 'orange', 'orange red', 
        'orchid', 'pale golden rod', 'pale green', 'pale turquoise', 'pale violet red', 
        'papaya whip', 'peach puff', 'peru', 'pink'
    ]

    color_pattern = r'\b(' + '|'.join(colors) + r')\b'
    brand = re.sub(color_pattern, '', brand, flags=re.IGNORECASE).strip()
    return brand

flipkart['brand'] = flipkart['brand'].apply(remove_colors)

Since there are unnecessary values in the `product_category_tree` column, we will be extracting the main information from the column and storing it in a list. 

In [5]:
# changing the 'product_category_tree' column

flipkart['product_category_tree'] = flipkart['product_category_tree'].map(lambda x:x.strip('[]'))
flipkart['product_category_tree'] = flipkart['product_category_tree'].map(lambda x:x.strip('""'))
flipkart['product_category_tree'] = flipkart['product_category_tree'].map(lambda x:x.split('>>'))

def clean_list(categories):
    cleaned_categories = [category.strip().replace('"', "'") for category in categories]
    return cleaned_categories

flipkart['product_category_tree'] = flipkart['product_category_tree'].apply(clean_list)


We thought it would be more informative for the product database to have a column on the percentage of discount of the item.

In [6]:
flipkart['percentage_discount'] = ((flipkart['retail_price'] - flipkart['discounted_price']) / flipkart['retail_price']) * 100
flipkart

,uniq_id,product_name,product_category_tree,pid,retail_price,discounted_price,discount,description,overall_rating,brand,product_specifications,percentage_discount
0,c2d766ca982eca8304150849735ffef9,alisha solid women's cycling shorts,"[clothing, women's clothing, lingerie, sleep &...",srteh2ff9kedefgf,999.0,379.0,62.06,key features of alisha solid women's cycling s...,no rating available,alisha,number of contents in sales package: pack of 3...,62.062062
1,7f7036a6d550aaa89d34c77bd39a5e48,fabhomedecor fabric double sofa bed,"[furniture, living room furniture, sofa beds &...",sbeeh3qgu7mfyjfy,32157.0,22646.0,29.58,fabhomedecor fabric double sofa bed (finish co...,no rating available,fabhomedecor,installation & demo details: installation and ...,29.576764
2,f449ec65dcbc041b6ae5e6a32717d01b,aw bellies,"[footwear, women's footwear, ballerinas, aw be...",shoeh4grsubjgzxe,999.0,499.0,50.05,key features of aw bellies sandals wedges heel...,no rating available,aw,ideal for: women occasion: casual color: red o...,50.050050
3,0973b37acd0c664e3de26e97e5571454,alisha solid women's cycling shorts,"[clothing, women's clothing, lingerie, sleep &...",srteh2f6huzmq6sj,699.0,267.0,61.8,key features of alisha solid women's cycling s...,no rating available,alisha,number of contents in sales package: pack of 2...,61.802575
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,sicons all purpose arnica dog shampoo,"[pet supplies, grooming, skin & coat care, sha...",psoeh3zydmsyarj5,220.0,210.0,4.55,specifications of sicons all purpose arnica do...,no rating available,sicons,pet type: dog brand: sicons quantity: 500 ml m...,4.545455
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,7179d2f6c4ad50a17d014ca1d2815156,walldesign small vinyl sticker,"[baby care, baby & kids gifts, stickers, walld...",stie7kfjakstdy9g,1500.0,730.0,51.33,buy walldesign small vinyl sticker for rs.730 ...,no rating available,walldesign,number of contents in sales package: pack of 5...,51.333333
19996,71ac419198359d37b8fe5e3fffdfee09,wallmantra large vinyl stickers sticker,"[baby care, baby & kids gifts, stickers, wallm...",stie9f5urnqgjcgh,1429.0,1143.0,20.01,buy wallmantra large vinyl stickers sticker fo...,no rating available,wallmantra,number of contents in sales package: pack of 1...,20.013996
19997,93e9d343837400ce0d7980874ece471c,elite collection medium acrylic sticker,"[baby care, baby & kids gifts, stickers, elite...",stie7vaydkqzebsd,1299.0,999.0,23.09,buy elite collection medium acrylic sticker fo...,no rating available,elite collection,number of contents in sales package: pack of 1...,23.094688
19998,669e79b8fa5d9ae020841c0c97d5e935,elite collection medium acrylic sticker,"[baby care, baby & kids gifts, stickers, elite...",stie8ysveppcz42y,1499.0,1199.0,20.01,buy elite collection medium acrylic sticker fo...,no rating available,elite collection,number of contents in sales package: pack of 1...,20.013342




As the Synthetic Dataset is based off the FlipKart Dataset, we will be using the cleaned version of the FlipKart Dataset for the Synthetic Dataset.
The new datasets can be found in the newData Folder. 

In [7]:
# saving the cleaned data
flipkart.to_csv('newData/flipkart_cleaned.csv', index=False)